In [1]:
import csv
import json
import tensorflow as tf
import numpy as np
import urllib


def solution_model():

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy') > 0.9):
                print("Reached 99% accuracy so cancelling training!")
                self.model.stop_training = True

    #callbacks = myCallback()
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
    urllib.request.urlretrieve(url, 'sarcasm.json')

    # DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    training_size = 20000

    sentences = []
    labels = []

    # YOUR CODE HERE
    gpus = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(gpus[0], enable=True)
    '''
    if gpus:
        try:
            tf.config.experimental.set_virtual_device_configuration(
                gpus[0],
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1200)]
            )
        except RuntimeError as e:
            print(e)
    '''
    tf.keras.backend.clear_session()
    print("Num GPUs:", len(tf.config.list_physical_devices('GPU')))

    early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,
                                                      monitor='val_loss')
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     min_lr=1e-5,
                                                     patience=5,
                                                     mode='min')
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_accuracy',
                                                          filepath='saved_models/best_model.h5',
                                                          save_best_only=True,
                                                          verbose=1)
    callbacks = [
        myCallback(),
        #early_stopping,
        reduce_lr,
        model_checkpoint
    ]

    with open('sarcasm.json', 'r') as file:
        data = json.load(file)
        for row in data:
            sentences.append(row['headline'])
            labels.append(row['is_sarcastic'])

    # Prepare data
    train_sentences = sentences[:training_size]
    valid_sentences = sentences[training_size:]

    train_labels = labels[:training_size]
    valid_labels = labels[training_size:]

    # Prepare input
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size,
                                                      oov_token=oov_tok)
    tokenizer.fit_on_texts(train_sentences)

    train_seqs = tokenizer.texts_to_sequences(train_sentences)
    train_pad_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs,
                                                                   maxlen=max_length,
                                                                   padding=padding_type,
                                                                   truncating=trunc_type)
    valid_seqs = tokenizer.texts_to_sequences(valid_sentences)
    valid_pad_seqs = tf.keras.preprocessing.sequence.pad_sequences(valid_seqs,
                                                                   maxlen=max_length,
                                                                   padding=padding_type,
                                                                   truncating=trunc_type)

    train_labels = np.array(train_labels)
    valid_labels = np.array(valid_labels)

    # Training
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size+1,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  #weights=[embeddings_matrix],
                                  trainable=True),

        # tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'),
        # tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'),
        # tf.keras.layers.MaxPooling1D(pool_size=2),
        # tf.keras.layers.SpatialDropout1D(0.2),
        # tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
        # tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
        # tf.keras.layers.GlobalAveragePooling1D(),
        # tf.keras.layers.Dense(units=128, activation='relu'),

        # tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
        # tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
        # tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
        # tf.keras.layers.GlobalAveragePooling1D(),
        # tf.keras.layers.Dense(units=128, activation='relu'),

        # tf.keras.layers.Dropout(0.2),
        # tf.keras.layers.GlobalAveragePooling1D(),
        # tf.keras.layers.Dense(units=128, activation='relu'),

        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        # YOUR CODE HERE. KEEP THIS OUTPUT LAYER INTACT OR TESTS MAY FAIL
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    print(model.summary())
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  metrics=['accuracy'])

    model.fit(x=train_pad_seqs,
              y=train_labels,
              validation_data=(valid_pad_seqs, valid_labels),
              batch_size=128,
              shuffle=True,
              epochs=25,
              callbacks=callbacks)

    return model

# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("saved_models/examQ4.h5")



2022-10-15 16:52:53.138670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-15 16:52:53.217928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-15 16:52:53.218178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Num GPUs: 1


2022-10-15 16:52:54.163689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 16:52:54.190134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-15 16:52:54.190495: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-15 16:52:54.190701: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           16016     
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                1560      
                                                                 
 dropout (Dropout)           (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 30,145
Trainable params: 30,145
Non-trainable params: 0
____________________________________________________

2022-10-15 16:52:59.823852: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


157/157 [==============================] - ETA: 0s - loss: 0.4654 - accuracy: 0.7704
Epoch 1: val_accuracy improved from -inf to 0.60292, saving model to saved_models/best_model.h5
157/157 [==============================] - 11s 28ms/step - loss: 0.4654 - accuracy: 0.7704 - val_loss: 0.6176 - val_accuracy: 0.6029 - lr: 0.0100
Epoch 2/25
156/157 [============================>.] - ETA: 0s - loss: 0.4085 - accuracy: 0.8083
Epoch 2: val_accuracy improved from 0.60292 to 0.82188, saving model to saved_models/best_model.h5
157/157 [==============================] - 4s 25ms/step - loss: 0.4083 - accuracy: 0.8084 - val_loss: 0.3843 - val_accuracy: 0.8219 - lr: 0.0100
Epoch 3/25
155/157 [============================>.] - ETA: 0s - loss: 0.3513 - accuracy: 0.8456
Epoch 3: val_accuracy did not improve from 0.82188
157/157 [==============================] - 4s 26ms/step - loss: 0.3515 - accuracy: 0.8456 - val_loss: 0.3764 - val_accuracy: 0.8214 - lr: 0.0100
Epoch 4/25
157/157 [=====================